In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import requests
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [2]:
model = GenerativeModel("gemini-pro")

In [3]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)

In [4]:
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

In [5]:
prompt = "What is the weather like in Boston?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.09521484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.08251953125
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1484375
    severity: HARM_SEVERITY_LOW
    severity_score: 0.2060546875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.140625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.08154296875
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    prob

In [6]:
response.candidates[0].content.parts[0].function_call

name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "Boston"
    }
  }
}

In [7]:
get_location = FunctionDeclaration(
    name="get_location",
    description="Get latitude and longitude for a given location",
    parameters={
    "type": "object",
    "properties": {
        "poi": {
            "type": "string",
            "description": "Point of interest"
        },
        "street": {
            "type": "string",
            "description": "Street name"
        },
        "city": {
            "type": "string",
            "description": "City name"
        },
        "county": {
            "type": "string",
            "description": "County name"
        },
        "state": {
            "type": "string",
            "description": "State name"
        },
        "country": {
            "type": "string",
            "description": "Country name"
        },
        "postal_code": {
            "type": "string",
            "description": "Postal code"
        },
    },
},
)
location_tool = Tool(
    function_declarations=[get_location],
)

In [8]:
prompt = """
I want to get the lat/lon coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043, US
"""
response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[location_tool],
)
response.candidates[0].content.parts[0]

function_call {
  name: "get_location"
  args {
    fields {
      key: "city"
      value {
        string_value: "Mountain View"
      }
    }
    fields {
      key: "country"
      value {
        string_value: "US"
      }
    }
    fields {
      key: "postal_code"
      value {
        string_value: "94043"
      }
    }
    fields {
      key: "state"
      value {
        string_value: "CA"
      }
    }
    fields {
      key: "street"
      value {
        string_value: "1600 Amphitheatre Pkwy"
      }
    }
  }
}

In [9]:
response.candidates[0].content.parts[0].function_call.args

{'state': 'CA',
 'postal_code': '94043',
 'street': '1600 Amphitheatre Pkwy',
 'city': 'Mountain View',
 'country': 'US'}

In [10]:
x = response.candidates[0].content.parts[0].function_call.args

url = "https://nominatim.openstreetmap.org/search?"
for i in x:
    url += '{}="{}"&'.format(i, x[i])
url += "format=json"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (kHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
x = requests.get(url, headers=headers)
content = x.json()
content

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
get_product_info_func = FunctionDeclaration(
    name="get_product_sku",
    description="Get the SKU for a product",
    parameters={
    "type": "object",
    "properties": {
        "product_name": {
            "type": "string",
            "description": "Product name"
        }
    }
},
)
get_store_location_func = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)
place_order_func = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
    "type": "object",
    "properties": {
        "product": {
            "type": "string",
            "description": "Product name"
        },
        "account": {
            "type": "integer",
            "description": "Account number"
        },
        "address": {
            "type": "string",
            "description": "Shipping address"
        }
    }
},
)
retail_tool = Tool(
    function_declarations=[get_product_info_func, 
                           get_store_location_func, 
                           place_order_func,
                          ],
)

In [12]:
model = GenerativeModel("gemini-pro", 
                        generation_config={"temperature": 0},
                        tools=[retail_tool])
chat = model.start_chat()

In [13]:
prompt = """
Do you have the Pixel 8 Pro in stock?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_sku"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

In [14]:
response.candidates[0].content.parts[0].function_call.args

{'product_name': 'Pixel 8 Pro'}

In [15]:
api_response = {"sku": "GA04834-US", "in_stock": "yes"}

In [16]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_sku",
        response={
            "content": api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, the Pixel 8 Pro is in stock. The SKU is GA04834-US."

In [17]:
prompt = """
Where can I buy one near Mountain View, CA?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

In [18]:
response.candidates[0].content.parts[0].function_call.args

{'location': 'Mountain View, CA'}

In [19]:
api_response = {"store": "1600 Amphitheatre Pkwy, Mountain View, CA 94043, US"}

In [20]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "The closest store to Mountain View, CA is located at 1600 Amphitheatre Pkwy, Mountain View, CA 94043, US."

In [21]:
prompt = """
I’d like to place an order for a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089, using account number 101.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "place_order"
  args {
    fields {
      key: "account"
      value {
        number_value: 101.0
      }
    }
    fields {
      key: "address"
      value {
        string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
      }
    }
    fields {
      key: "product"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

In [23]:
response.candidates[0].content.parts[0].function_call.args

{'address': '1155 Borregas Ave, Sunnyvale, CA 94089',
 'account': 101,
 'product': 'Pixel 8 Pro'}

In [24]:
api_response = {"payment_status": "paid", "order_number": 12345, "est_arrival": "2 days"}

In [25]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Your order has been placed. The estimated arrival time is 2 days. Your order number is 12345 and the payment status is paid.\n"